In [79]:
import json
import os
from PIL import Image
import pickle
import numpy as np
import cv2

def getLatestTrack():
    dir_ = "website/uploads/"
    file = sorted(os.listdir(dir_))[-1]
    return os.path.join(dir_, file)
getLatestTrack()

imagesDir = "website/image_uploads/"
ultrasoundDir = "../../neuralnet/fastNNServer/outputs/"
class TrackedUS:
    def __init__(self, filename):
        self.track = np.array(pickle.load(open(getLatestTrack(), "rb"))).transpose()
        self.time = track[3]
        self.startTime = self.time[0]
        self.endTime = self.time[-1]
        
        #load camera pictures for tracking
        pictures = os.listdir(imagesDir)[1:]

        self.p_timestamps = []

        self.p_arrays = []
        for p in pictures:
            t_float = float(p[:-4])
            if self.startTime < t_float < self.endTime:
                self.p_timestamps.append(t_float)
                self.p_arrays.append(np.rot90(np.array(Image.open(imagesDir + p)), 3))
        #load ultrasound images
        
        for us in os.listdir(ultrasoundDir):
            usStart, usEnd = [float(timeStamp) for timeStamp in us.split("_")]
            
            if usStart < self.startTime < self.endTime < usEnd:
                print("yay")
                self.ultrasound = pickle.load(open(ultrasoundDir + us, "rb"))
                break
        else:
            raise Exception("no ultrasound found that covers the time of the track")
    def getAttitudeFromTimestamp(self, t):
        return [np.interp([t], track[3], track[0]), 
                np.interp([t], track[3], track[1]),
                np.interp([t], track[3], track[2])]
    
    def playUS(self):
        for el in self.ultrasound:
            cv2.imshow("ultra", el[1])
            cv2.waitKey(10)
    def playVideo(self):
        for arr in self.p_arrays:
            cv2.imshow("ultra", arr)
            cv2.waitKey(10)
    def play(self):
        us_index = 0
        video_index = 0
        try:
            while True:
                if self.p_timestamps[video_index] < self.ultrasound[us_index][0]:
                    cv2.imshow("vid", self.p_arrays[video_index])
                    cv2.waitKey(10)
                    video_index += 1
                else:
                    cv2.imshow("ultra", self.ultrasound[us_index][1])
                    cv2.waitKey(10)
                    us_index += 1
        except IndexError:
            print(us_index, video_index)
        print(done)
    def play2(self):
        us_index = 0
        video_index = 0
        try:
            while True:
                
                if self.p_timestamps[video_index] < self.ultrasound[us_index][0]:
                    arr = self.p_arrays[video_index][::3, ::3]
                    v_timestamp = self.p_timestamps[video_index]
                    out = scipy.ndimage.rotate(arr, 
                        -self.getAttitudeFromTimestamp(v_timestamp)[1] * 360 / (2 * np.pi))
                    cv2.imshow("vid", out)
                    cv2.waitKey(10)
                    video_index += 1
                else:
                    arr = self.ultrasound[us_index][1][::3, ::3]
                    u_timestamp = self.ultrasound[us_index][0]
                    out = scipy.ndimage.rotate(arr, 
                        -self.getAttitudeFromTimestamp(u_timestamp)[1] * 360 / (2 * np.pi))
                    cv2.imshow("ultra", out)
                    cv2.waitKey(10)
                    us_index += 1
        except IndexError:
            print(us_index, video_index)
        print("done")

my_us = TrackedUS(getLatestTrack)
#my_us.playUS()
#my_us.playVideo()

yay


In [80]:
my_us.play2()


KeyboardInterrupt: 

In [77]:
cv2.waitKey?